note- i changed the location of notebook (after training and saving the model), so only ran relevant cells to load the saved model and tokenizer

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
import torch
import numpy as np

e:\IITB\Learner Space 2025\Intro-to-ML-and-NLP\hf_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
dataset = load_dataset("imdb")

small_train = dataset["train"].shuffle(seed=42).select(range(1000))  # 1k samples
small_test = dataset["test"].shuffle(seed=42).select(range(250))     # 250 samples

In [8]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",  # Pad shorter sequences
        truncation=True,       # Cut sequences >512 tokens
        max_length=512         # BERT's max input length
    )

tokenized_dataset_train = small_train.map(tokenize, batched=True)
tokenized_dataset_test = small_test.map(tokenize, batched=True)

In [4]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        **metric.compute(predictions=predictions, references=labels),
        **f1_metric.compute(predictions=predictions, references=labels)
    }

In [15]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=2,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test,
    compute_metrics=compute_metrics,
)

In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.331382,0.872000,0.855856
2,No log,0.312415,0.872000,0.869919


e:\IITB\Learner Space 2025\Intro-to-ML-and-NLP\hf_env\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=126, training_loss=0.41880531916542657, metrics={'train_runtime': 4047.0763, 'train_samples_per_second': 0.494, 'train_steps_per_second': 0.031, 'total_flos': 526222110720000.0, 'train_loss': 0.41880531916542657, 'epoch': 2.0})

In [5]:
model = BertForSequenceClassification.from_pretrained("E:/IITB/Learner Space 2025/Intro-to-ML-and-NLP/results/checkpoint-126")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

model.save_pretrained("E:/IITB/Learner Space 2025/Intro-to-ML-and-NLP/fine_tuned_bert")
tokenizer.save_pretrained("E:/IITB/Learner Space 2025/Intro-to-ML-and-NLP/fine_tuned_bert")

('E:/IITB/Learner Space 2025/Intro-to-ML-and-NLP/fine_tuned_bert\\tokenizer_config.json',
 'E:/IITB/Learner Space 2025/Intro-to-ML-and-NLP/fine_tuned_bert\\special_tokens_map.json',
 'E:/IITB/Learner Space 2025/Intro-to-ML-and-NLP/fine_tuned_bert\\vocab.txt',
 'E:/IITB/Learner Space 2025/Intro-to-ML-and-NLP/fine_tuned_bert\\added_tokens.json')

In [8]:
from transformers import pipeline

sample_text = "This movie was not fantastic!"
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)
print("Sample prediction:", pipe(sample_text)) # Label_1: Positive, Label_0: Negative

Device set to use cpu


Sample prediction: [{'label': 'LABEL_0', 'score': 0.9082422256469727}]
